In [1]:
from twikit import Client
import time
import json
from twikit import TwitterException 
import pandas as pd

# this API requires authentication
f = open('authentication.txt', 'r')
auth = f.read()
f.close()
auth_token = auth.split("\n")

# don't hardcode your email and password into something!!!
# the auth is in gitignore so I won't get hacked
USERNAME = str(auth_token[0])
EMAIL = str(auth_token[1])
PASSWORD = str(auth_token[2])

# Initialize client
client = Client(language='en-US', http2=True)

# Login to the service with provided user credentials
client.login(
    auth_info_1=USERNAME ,
    auth_info_2=EMAIL,
    password=PASSWORD
)

C:\Users\eaesp\AppData\Local\Temp\ipykernel_17824\2204781778.py:5: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


{'flow_token': 'g;171367316029157670:-1713673160348:v7o7wSpXEVV05latvxcFRvvH:14',
 'status': 'success',
 'subtasks': [{'subtask_id': 'LoginSuccessSubtask',
   'open_account': {'user': {'id': 1547081484695216130,
     'id_str': '1547081484695216130',
     'name': 'Eloragh Espie',
     'screen_name': 'EloraghEspie'},
    'next_link': {'link_type': 'subtask',
     'link_id': 'next_link',
     'subtask_id': 'SuccessExit'},
    'attribution_event': 'login'}},
  {'subtask_id': 'SuccessExit',
   'open_link': {'link': {'link_type': 'subtask',
     'link_id': 'next_link',
     'subtask_id': 'LoginOpenHomeTimeline'}}},
  {'subtask_id': 'LoginOpenHomeTimeline',
   'open_home_timeline': {'next_link': {'link_type': 'abort',
     'link_id': 'next_link'}}}]}

In [2]:
# Twitter LOVES to ban people when they log in repeatedly
# saving the cookies makes sure I don't get banned (often)

client.get_cookies()
client.save_cookies('cookies.json')
with open('cookies.json', 'r', encoding='UTF8') as f:
    client.set_cookies(json.load(f))

In [3]:
from twikit import UserUnavailable
from twikit import UserNotFound

def get_user_id(data_list):
    # load the cookies so you don't login a million times and get banned
    client.load_cookies('cookies.json')
    
    i = 0
    # pull IDs and insert them back into the list
    for item in data_list:
        handle = item[2]
        try:
            user_id = client.get_user_by_screen_name(handle).id
            item.insert(0, user_id)
            print(f'{i} Success {handle}, {user_id}')

        # if user is inaccessible these will keep the loop from breaking
        except UserUnavailable:
            print(f"{i }User {handle} unavailable")
        except UserNotFound:
            print(f'{i} User {handle} not found')

        i += 1
        time.sleep(5) # cooldown to avoid rate limits

    return(data_list)

In [4]:
def create_key(user_id, year):
    # load the cookies so you don't login a million times and get banned
    client.load_cookies('cookies.json')
    return(user_id + year)

In [5]:
import time
from math import ceil
import requests
from twikit import TooManyRequests
from twikit.utils import Endpoint

# this is a housekeeping function
# twitter API can throw rate limits
# they're kind of like timeouts
# this function just shows me how much longer I will be in timeout for

def get_limit_reset_time(endpoint: str):
    res = requests.get(
        endpoint,
        headers=client._base_headers,
        cookies=client.get_cookies()
    )
    return ceil(int(res.headers['x-rate-limit-reset']) - time.time())

In [6]:
# timeout check for accessing user IDs
try:
    print(client.get_user_by_screen_name('BarackObama'))
except TooManyRequests:
    
    reset_time = get_limit_reset_time(Endpoint.USER_TWEETS)
    print(f'rate limit is reset after {reset_time} seconds.')

<User id="813286">


In [27]:
data = pd.read_csv("Politicians.csv")
df = pd.DataFrame(data)

In [28]:
# turn pandas into list
# there is probably a better way to do this but this works 
data_list = []
for index, row in df.iterrows():

    row_list = df.loc[index, :].values.flatten().tolist()
    data_list.append(row_list)
    
print(data_list)

[[nan, 'Barack Obama', 'BarackObama'], [nan, 'Joe Biden', 'JoeBiden'], [nan, 'Dennis Kucinich', 'Dennis_Kucinich'], [nan, 'Bill Richardson', 'GovRichardson'], [nan, 'Mike Huckabee', 'GovMikeHuckabee'], [nan, 'Hillary Clinton', 'HillaryClinton'], [nan, 'Ron Paul', 'RonPaul'], [nan, 'John McCain', 'SenJohnMcCain'], [nan, 'Newt Gingrich', 'newtgingrich'], [nan, 'Mike Gravel', 'MikeGravel_US'], [nan, 'Alan Keyes', 'loyaltoliberty'], [nan, 'Mitt Romney', 'MittRomney'], [nan, 'Fred Thompson', 'fredthompson'], [nan, 'Chris Dodd', 'SenChrisDodd'], [nan, 'Rudy Guiliani', 'RudyGiuliani'], [nan, 'Ralph Nader', 'RalphNader'], [nan, 'Tom Tancredo', 'ttancredo'], [nan, 'Jill Stein', 'DrJillStein'], [nan, 'Virgil Goode', 'VirgilGoode'], [nan, 'Gary Johnson', 'GovGaryJohnson'], [nan, 'Rick Santorum', 'RickSantorum'], [nan, 'Hllary Clinton', 'HillaryClinton'], [nan, 'Bernie Sanders', 'SenSanders'], [nan, 'Donald Trump', 'realDonaldTrump'], [nan, 'Jeb Bush', 'JebBush'], [nan, 'Ted Cruz', 'TedCruz'], [na

In [29]:
batch1 = [item for item in data_list[0:50]]
batch2 = [item for item in data_list[50:100]]
batch3 = [item for item in data_list[100:150]]
batch4 = [item for item in data_list[150:]]

mass_data_list = [batch1, batch2, batch3, batch4]

In [30]:
for item in mass_data_list:
    print(item)

[[nan, 'Barack Obama', 'BarackObama'], [nan, 'Joe Biden', 'JoeBiden'], [nan, 'Dennis Kucinich', 'Dennis_Kucinich'], [nan, 'Bill Richardson', 'GovRichardson'], [nan, 'Mike Huckabee', 'GovMikeHuckabee'], [nan, 'Hillary Clinton', 'HillaryClinton'], [nan, 'Ron Paul', 'RonPaul'], [nan, 'John McCain', 'SenJohnMcCain'], [nan, 'Newt Gingrich', 'newtgingrich'], [nan, 'Mike Gravel', 'MikeGravel_US'], [nan, 'Alan Keyes', 'loyaltoliberty'], [nan, 'Mitt Romney', 'MittRomney'], [nan, 'Fred Thompson', 'fredthompson'], [nan, 'Chris Dodd', 'SenChrisDodd'], [nan, 'Rudy Guiliani', 'RudyGiuliani'], [nan, 'Ralph Nader', 'RalphNader'], [nan, 'Tom Tancredo', 'ttancredo'], [nan, 'Jill Stein', 'DrJillStein'], [nan, 'Virgil Goode', 'VirgilGoode'], [nan, 'Gary Johnson', 'GovGaryJohnson'], [nan, 'Rick Santorum', 'RickSantorum'], [nan, 'Hllary Clinton', 'HillaryClinton'], [nan, 'Bernie Sanders', 'SenSanders'], [nan, 'Donald Trump', 'realDonaldTrump'], [nan, 'Jeb Bush', 'JebBush'], [nan, 'Ted Cruz', 'TedCruz'], [na

In [26]:
i = 0
final_list = []

for list in mass_data_list:
    new_data_list = get_user_id(list)
    final_list += new_data_list
    time.sleep(120) # cool down to avoid rate limits


0 Success BarackObama, 813286
1 Success JoeBiden, 939091
2 Success Dennis_Kucinich, 15022633
3 Success GovRichardson, 31428685
4 Success GovMikeHuckabee, 15416505
5 Success HillaryClinton, 1339835893
6 Success RonPaul, 287413569
7 Success SenJohnMcCain, 19394188
8 Success newtgingrich, 20713061
9 Success MikeGravel_US, 752638969095999489
10 Success loyaltoliberty, 19637821
11 Success MittRomney, 50055701
12 Success fredthompson, 2704951
13 Success SenChrisDodd, 16317406
14 Success RudyGiuliani, 770781940341288960
15 Success RalphNader, 77314692
16 Success ttancredo, 64534908
17 Success DrJillStein, 111216929
18 Success VirgilGoode, 605641396
19 Success GovGaryJohnson, 95713333
20 Success RickSantorum, 58379000
21 Success HillaryClinton, 1339835893
22 Success SenSanders, 29442313
23 Success realDonaldTrump, 25073877
24 Success JebBush, 113047940
25 Success TedCruz, 23022687
26 Success MarcoRubio, 15745368
27 Success JohnKasich, 18020081
28 Success SenBobCorker, 73303753
29 Success Mike_

KeyboardInterrupt: 

In [18]:
import sqlite3
conn = sqlite3.connect('tweets.db')
c = conn.cursor()

c.execute("""CREATE TABLE politicians (
          twitter_user_id INTEGER primary key,
          politician_name STRING,
          twitter_handle STRING
          )""")

In [17]:
conn.execute('DROP TABLE politicians')

In [ ]:
c.executemany("INSERT INTO tweets VALUES (?,?,?,?,?,?)", final_list)
conn.commit()